In [1]:
# Import modules
import math
import pandas as pd
import numpy as np
import pandas_datareader as pdr
import yfinance as yf
import matplotlib.pyplot as plt
import os
from time import time
from dateutil.relativedelta import relativedelta
from datetime import datetime

In [340]:
def convert_hdb_index_to_daily(file_name):
    df = pd.read_csv(file_name, parse_dates=['quarter'])
    df = df.rename({"quarter": "quarter_start"}, axis=1)
    # assuming index column indicates the index price of Resale HDBs on the LAST DAY of the quarter.

    year_20_21_q3 = df[122:] # we only need dates from 2020-10-01 for our analysis but retrieving from 
    year_20_21_q3["price_increase_prev_quarter"] = year_20_21_q3["index"].diff(1)
    year_20_21_q3["quarter_end"] = pd.offsets.QuarterEnd() + year_20_21_q3["quarter_start"] 
    year_20_21_q3["date"] = year_20_21_q3["quarter_end"]

    year_20_21_q3_quartered = year_20_21_q3.set_index("date")
    year_20_21_q3_quartered.index = year_20_21_q3_quartered.index.to_period('Q')

    daily_df = year_20_21_q3_quartered.resample('D').ffill()
    idx = daily_df.index[-1] + pd.offsets.Day(1)
    daily_df = daily_df.append(daily_df.iloc[[-1]].rename({daily_df.index[-1]: idx}))

    daily_df.index = daily_df.index.to_timestamp()

    hist_benchmark = pdr.get_data_yahoo("\^STI", start = "2020-09-30", end = "2021-09-30")
    indices = list(hist_benchmark.index)

    # daily_df = daily_df.loc[indices]
    # return daily_df

    # #resample with forward filling values, remove last helper row
    df1 = daily_df.resample('D').ffill().iloc[:-1]

    df1 = df1.loc[indices]
    # #divide by size of quarters
    # df1['daily_price_increase'] = daily_df.apply(lambda x: x.price_increase_prev_quarter/daily_df.groupby("quarter_start")["index"].count()[x.quarter_start], axis=1)

    df1['daily_price_increase'] = df1.apply(lambda x: x.price_increase_prev_quarter/df1.groupby("quarter_start")["index"].count()[x.quarter_start], axis=1)

    #only take the relevant quarters for 1 year
    df1_cut = df1["2020-09-30":]
    df1_cut["price"] = 0
    df1_cut.at["2020-09-30", "price"] = df1_cut.at["2020-09-30","index"] # set price on last day of 2020 q3 to index price
    df_idx = df1_cut.reset_index()

    for i in range(1, len(df_idx)):
        df_idx.loc[i, 'price'] = df_idx.loc[i-1, 'price'] + df_idx.loc[i, 'daily_price_increase']

    df_idx["Benchmark Returns"] = np.log(df_idx["price"] / df_idx["price"].shift(1))
    df_final = df_idx[["date", "price", "Benchmark Returns"]]
    df_final = df_final.set_index("date").rename({"price": "Close_Price"}, axis=1)

    # df_final.index = df_final.index.to_timestamp()

    return df_final

In [341]:
df = convert_hdb_index_to_daily("housing-and-development-board-resale-price-index-1q2009-100-quarterly.csv")

/Users/limjohn/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/limjohn/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/limjohn/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [342]:
df

,Close_Price,Benchmark Returns
date,,
2020-09-30,133.900000,NaN
2020-10-01,133.964615,0.000482
2020-10-02,134.029231,0.000482
2020-10-05,134.093846,0.000482
2020-10-06,134.158462,0.000482
...,...,...
2021-09-24,150.337500,0.000437
2021-09-27,150.403125,0.000436
2021-09-28,150.468750,0.000436
